In [1]:
import sys
sys.path.append('../zoho_data_app')

import main_module as m

import pandas as pd
import gspread
import re
import datetime

import dateutil

%load_ext autoreload
%autoreload 2


# file_loc = r'C:\Users\fajar.fatoni\Documents\Python\Data'
file_loc = r'C:\Users\fajar.fatoni\Documents\Python\Data'

gc = gspread.oauth(
    credentials_filename= r'C:\Users\fajar.fatoni\Documents\Python\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID_SOURCE = '1PiRLPCroErrgZ07dlLdZZxUKzFmSZNnFwLy48MDmK2A'


In [2]:
sheetname = 'Data'

# Open the spreadsheet
spreadsheet_source = gc.open_by_key(SHEET_ID_SOURCE)
worksheet = spreadsheet_source.worksheet(sheetname)
rows = worksheet.get_all_records()
result = pd.DataFrame(rows)
result

,number,ContactID,FirstName,LastName,AddressLine1,AddressLine2,Town,City,Country,Post Code,...,WeddingDate,Keywords,PanCardNo,Account Type,Account Status,OdysseyNo,Servicing Office,club,Account Currency,Sheet Name
0,1000056,921301,Jane,Chrisdianty,WELLINGTON RESIDENCE BLOK A1,"RT. 009/009 BENDA BARU, PAMULANG",TANGERANG SELATAN,BANTEN,Indonesia,-,...,,,,KARMA CLUB,Active,C82046362,KARMA CONCIERGE BALI,Karma Club,,KCB-ID-Excluded KCID
1,1000056,921274,Yohanes,Mustamu,WELLINGTON RESIDENCE BLOK A1,"RT. 009/009 BENDA BARU, PAMULANG",TANGERANG SELATAN,BANTEN,Indonesia,15417,...,,,,KARMA CLUB,Active,C82046362,KARMA CONCIERGE BALI,Karma Club,,KCB-ID-Excluded KCID
2,1000109,921347,Willy Stevan Berny,Suak,"JL. KEMBANG NO 59, SARIO KOTA BARU",,MANADO,SULAWESI UTARA,Indonesia,95114,...,1998-08-20,,,KARMA CLUB,Active,C82053141,KARMA CONCIERGE BALI,Karma Club Indonesia,,KCID-ID
3,1000109,921354,Cecilia Ingrid,Tjan,"JL. KEMBANG NO 59, SARIO KOTA BARU",,MANADO,SULAWESI UTARA,Indonesia,95114,...,,,,KARMA CLUB,Active,C82053141,KARMA CONCIERGE BALI,Karma Club Indonesia,,KCID-ID
4,1000121,921368,Muhammad,Mustaqin,"JL. MELATI NO. 5 RT.006/007 RAGUNAN, PASAR MINGGU",,JAKARTA SELATAN,DKI JAKARTA,Indonesia,12550,...,,,,KARMA CLUB,Active,C82052905,KARMA CONCIERGE BALI,Karma Club,,KCB-ID-Excluded KCID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85441,109144,6365,Ronak Pravinbhai,Shah,"403, EMERALD RESIDENCY3, OPP. KRUPA PETROL PUMP","AMBICA DAIRY WALI LANE, NR. PARIMAL GARDEN CRO...",AHMEDABAD,GUJARAT,India,380007,...,2018-03-04,,,KARMA CLUB,Active,C82031493,KARMA CONCIERGE INDIA,Karma Club India,INR,KCI-India
85442,38994,43114,Marie Lourdes,Peres,"MARIE - LOU HAVEN, OPP. PANCHAYAT","MADUNGO, ASSAGAO",BARDEZ,GOA,India,403507,...,,,,KARMA CLUB,Active,C82008192,KARMA CONCIERGE INDIA,Karma Club India,INR,KCI-India
85443,68873,74738,S,Ravindran,"14/102, FIRST FLOOR, LAKSHMI NIVAS","22nd MAIN ROAD, PADMANABHANAGAR",BENGALURU,KARNATAKA,India,560070,...,2019-01-30,,ACKPR5580Q,KARMA ROYAL RESIDENCES,Active,D89100854,KARMA CONCIERGE INDIA,KRR Haathi Mahal,INR,KCI-India
85444,132366,15009,Ann,Gilchrist,HAWBERRY HOUSE,CLARENDON ROAD,MANCHESTER,MANCHESTER,England,,...,,,Special Waiver,KARMA CLUB,Active,C82045914,KARMA CONCIERGE INDIA,,,KCI-Non India


In [3]:
result.columns

Index(['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
       'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email',
       'Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Keywords', 'PanCardNo', 'Account Type',
       'Account Status', 'OdysseyNo', 'Servicing Office', 'club',
       'Account Currency', 'Sheet Name'],
      dtype='object')

In [4]:
result['Main DB'] = ''
result['Sub DB'] = ''

result.loc[result['Sheet Name'] == 'HOK', 'Main DB'] = 'HOK'
result.loc[result['Sheet Name'] == 'KC 2021', ['Main DB','Sub DB']] = ['KC 2021','KCEU']
result.loc[result['Sheet Name'] == 'KCB-Elite-ENG', ['Main DB','Sub DB']] = ['KCB', 'Elite-Eng']
result.loc[result['Sheet Name'] == 'KCB-ID-Excluded KCID', ['Main DB','Sub DB']] = ['KCB', 'ID-Exc']
result.loc[result['Sheet Name'] == 'KCB-NH-ID', ['Main DB','Sub DB']] = ['KCB', 'ID-NH']
result.loc[result['Sheet Name'] == 'KCB-NH-ENG', ['Main DB','Sub DB']] = ['KCB', 'ENG-NH']
result.loc[result['Sheet Name'] == 'KCB-ENG-Excluded KCID', ['Main DB','Sub DB']] = ['KCB', 'ENG-Exc']
result.loc[result['Sheet Name'] == 'KCB-St.Martin', ['Main DB','Sub DB']] = ['KCB', 'ST.Martin-Elite']
result.loc[result['Sheet Name'] == 'KCE', ['Main DB']] = ['KCDEU']
result.loc[result['Sheet Name'] == 'KCI-Elite-Non India', ['Main DB','Sub DB']] = ['KCI', 'Elite-Non India']
result.loc[result['Sheet Name'] == 'KCI-India', ['Main DB','Sub DB']] = ['KCI', 'India']
result.loc[result['Sheet Name'] == 'KCI-NH-India', ['Main DB','Sub DB']] = ['KCI', 'NH-India']
result.loc[result['Sheet Name'] == 'KCI-NH-Non India', ['Main DB','Sub DB']] = ['KCI', 'NH-Non India']
result.loc[result['Sheet Name'] == 'KCI-Non India', ['Main DB','Sub DB']] = ['KCI', 'Non India']
result.loc[result['Sheet Name'] == 'KCID-ID', ['Main DB','Sub DB']] = ['KCID', 'ID']
result.loc[result['Sheet Name'] == 'KCID-ENG', ['Main DB','Sub DB']] = ['KCID', 'ENG']

In [5]:
current_date = datetime.date.today()

formatted_date = current_date.strftime("%d-%m-%Y")
result['Date DB'] = formatted_date

result['Tags'] = 'Members'

In [6]:
# if data dated is string
# using error=cource to prevent N/A value errors
result['DoB'] = pd.to_datetime(result['DoB'], format='mixed', errors='coerce')
result['DoB'] = result['DoB'].dt.strftime("%d %b %Y")

result['WeddingDate'] = pd.to_datetime(result['WeddingDate'], format='mixed', errors='coerce')
result['WeddingDate'] = result['WeddingDate'].dt.strftime("%d %b %Y")

In [7]:
result['DoB']

0        03 Jan 1973
1        23 Aug 1978
2        10 Jan 1971
3        19 Sep 1973
4        03 Nov 1971
            ...     
85441    09 Mar 1969
85442            NaN
85443            NaN
85444            NaN
85445            NaN
Name: DoB, Length: 85446, dtype: object

In [8]:
# result.columns
# result.columns = ['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
#        'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email',
#        'Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
#        'WeddingDate', 'Account Type', 'Account Status', 'Servicing Office', 
#        'Keywords', 'PanCardNo', 'OdysseyNo', 'segment',
#        'Account Currency', 'Main DB', 'Sub DB', 'Date DB', 'Tags']

In [9]:
result.rename(columns={
    'segment': 'Member Segment',
}, inplace=True)

In [10]:
result.fillna('', inplace=True)

In [11]:
result['Email Old'] = result['Email']

m.clean_space(result, 'Email')
m.email_correction(result, 'Email')

result['Email Correction'] = 'No'
result.loc[result['Email Old']!=result['Email'], 'Email Correction'] = 'Yes'

c:\Users\fajar.fatoni\Documents\Python\Task Offline\member_app\../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\member_app\../zoho_data_app\main_module.py:271: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediat

In [12]:
result = result.replace(r'\n', '', regex=True)

In [13]:
result[['Email','Main DB']].loc[result['Email'].str.contains(',', na=False)]

,Email,Main DB


In [14]:
result.columns

Index(['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
       'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email',
       'Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Keywords', 'PanCardNo', 'Account Type',
       'Account Status', 'OdysseyNo', 'Servicing Office', 'club',
       'Account Currency', 'Sheet Name', 'Main DB', 'Sub DB', 'Date DB',
       'Tags', 'Email Old', 'Email Correction'],
      dtype='object')

In [15]:
result = result.reindex(columns=['number', 'ContactID', 'FirstName', 'LastName', 'AddressLine1',
       'AddressLine2', 'Town', 'City', 'Country', 'Post Code', 'Email Old','Email',
       'Email Correction','Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Account Type', 'Account Status', 'Servicing Office',
       'Sheet Name', 'Keywords', 'PanCardNo', 'OdysseyNo',
       'Account Currency', 'Main DB', 'Sub DB', 'Date DB', 'Tags'])

In [16]:
result.rename(columns={
    'Town': 'State',
    'number': 'Number'
    # Add other renames as needed...
}, inplace=True)

In [17]:
result.to_excel(file_loc+'\compiled-member.xlsx', index=False)
# result.to_excel(file_loc+'\compiled-member-test.xlsx', index=False)

In [18]:
result = result.reindex(columns=['Number', 'ContactID', 'FirstName', 'LastName', 'State', 'City', 'Country', 'Post Code','Email',
       'Email Correction','Phones', 'Owner', 'Primary', 'Family', 'Nationality', 'DoB',
       'WeddingDate', 'Account Type', 'Account Status', 'Servicing Office', 'Keywords', 'PanCardNo', 'OdysseyNo',
       'Account Currency', 'Main DB', 'Sub DB', 'Date DB', 'Tags'])

In [19]:
# row_split = 30000
# file_name = 'compiled-member'
# file_type = 'csv'

# m.split_file(result, row_split, file_name, file_type, file_loc)

result.to_csv(file_loc+'\compiled-member.csv', index=False)
# result.to_csv(file_loc+'\compiled-member-test.csv', index=False)

In [20]:
# result.loc[result['Email'] == 'elliotstyche@hotmail.co.uk'].to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\test-upload.csv', index=False)